In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report

In [2]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
print(bad[2])
print(true[2])

Пояним эту мысль.
Поясним эту мысль


In [4]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [5]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


In [6]:
mistakes = []
total = 0

for i in range(len(true)):
    
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        
        total += 1

In [7]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.13034358769476628


In [8]:
mistakes[:5]

[('симпатичнейшее', 'симпатичнейшое'),
 ('апофеозом', 'опофеозом'),
 ('поясним', 'пояним'),
 ('получатся', 'полчатся'),
 ('очень', 'оччччень')]

In [9]:
Counter(mistakes).most_common(10)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('вообще', 'ваще'), 17),
 (('естественно', 'естесственно'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9)]

In [10]:
corpus = open('corpus_ng.txt', encoding='utf8').read().splitlines()

In [11]:
from difflib import get_close_matches

In [12]:
import textdistance

In [13]:
def get_closest_match_with_metric(text, lookup, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(1)[0]

In [14]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [15]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [16]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [17]:
def get_closest_match_vec(text, X, vec, TOPN=3):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [18]:
a = get_closest_match_vec('опофеоз', X, vec)
print(a)

['апофеоз', 'апофеозом', 'апофеоза']


In [19]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:3]
    words = [id2word[top] for top in topn]
    
    similarities = Counter()
    for word in words:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    closest = similarities.most_common(1)[0][0]
    return closest

In [20]:
a = get_closest_hybrid_match('опофеоз', X, vec)
print(a)

апофеоз


In [39]:
mistakes = []
total = 0
true_part = true[:10]
bad_part = bad[:10]

for i in range(len(true_part)):
    bad_or_not = get_closest_hybrid_match(bad_part[i], X, vec, metric=textdistance.levenshtein)
    word_pairs = align_words(true_part[i], bad_part[i])
    for pair in word_pairs:
        func_word = get_closest_hybrid_match (pair[1], X, vec, metric=textdistance.levenshtein)
        if pair[0] != func_word:unc_word)
            mistakes.append([pair[1], func_word])
        
        total += 1

In [40]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.25


In [41]:
print (mistakes)

[['симпатичнейшое', 'пластичнейшими'], ['шпионское', 'шпионские'], ['гламурный', 'гуманный'], ['бонда', 'банд'], ['superheadz', 'herederas'], ['clap', 'place'], ['camera', 'america'], ['полчатся', 'ополчатся'], ['язычки', 'язычка'], ['все', 'есв'], ['оччччень', 'чечни'], ['милые', 'милы'], ['нащщот', 'защищено'], ['чавеса', 'чавес'], ['на', 'анан'], ['попавшим', 'пропавшим'], ['аварийно-спасательных', 'аварийно-восстановительных'], ['нарасно', 'роснано'], ['на', 'анан'], ['вобщем', 'общем'], ['как', 'как'], ['вы', 'вы'], ['знаете', 'знаете'], ['из', 'из'], ['моего', 'моего'], ['не', 'не'], ['давнего', 'давнего'], ['на', 'анан'], ['пропажу', 'пожару'], ['на', 'анан'], ['почте.ру', 'т.п']]


По ошибкам видно, что он плохо распознает слова, которые совсем не похожи на свою правильную пару (в-вобщем, очень-очччень). 